# Nodding

This notebook shows the reduction path of a nodding observation in a multi-beam receiver.

Similar to a position-switched observation, two beams are selected for simultaneous observing (though the receiver can have more than two beams). In the first scan BEAM1 is looking at the source, while BEAM2 is looking at an assumed OFF position. In the next scan, BEAM2 will be looking at the source, while BEAM1 is looking at (another) OFF position. This will result in two position-switched solutions, which are then averaged for the final spectrum. 

The obvious big advantage of this observing mode is that the telescope is always ON source, bringing the noise down by $\sqrt{2}$, minus a small amount of slewing time.

This notebook also includes the matching getps() observation from another scan so we can confirm the improvement in signal/noise.


In [ ]:
# %matplotlib widget

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy import units as u
from dysh.fits.gbtfitsload import GBTFITSLoad
from dysh.util.files import dysh_data
from pathlib import Path
from dysh.util.download import from_url

In [ ]:
# filename = dysh_data(example="nod-KFPA/data/TGBT22A_503_02.raw.vegas")
# filename = dysh_data(example="nod-KFPA/data/TGBT22A_503_02.raw.vegas.trim7.fits")

In [ ]:
url = "http://www.gb.nrao.edu/dysh/example_data/nod-KFPA/data/TGBT22A_503_02.raw.vegas.trim.fits"
savepath = Path.cwd() / "data"
filename = from_url(url, savepath)

This trimmed dataset is an extraction from a much larger dataset (19GB) and can take some time to load if it's the first time.

In [ ]:
sdf = GBTFITSLoad(filename)

In [ ]:
sdf.summary()

We will use scans 60 and 61 for the position-switching (PS) observation.

Also note for the 7-beam KFPA receiver, the central beam (fdnum=0) will be the source tracking beam.

In [ ]:
sp0 = sdf.getps(scan=60,plnum=0,ifnum=0,fdnum=0).timeaverage()

In [ ]:
sp0s = sp0.smooth('box',51)
sp0s.plot(title='dysh getps',xaxis_unit="chan")
sp0s.stats()

Now the nodding method, using scans 62 and 63.  This will return one scanblock with two NodScan's, each representing a "PS" observations that need to be averaged to get the final spectrum.  We can do that with the timeaverage() function directly applying to this list of (two) NodScan's.  And for good measure we will smooth it a little bit as we did with the previous PS observation.

Normally the FITS header will contain information on which is BEAM1 and which is BEAM2. In this case beams 2 and 6 are the two nodding beams.

In [ ]:
sp2 = sdf.getnod(scan=[62,63],ifnum=0,plnum=0).timeaverage()
sp2s = sp2.smooth('box',51)
sp2s.stats()

In [ ]:
sp2s = sp2.smooth('box',51)
sp2s.plot(title='dysh getnod',xaxis_unit="chan")
print('getnod RMS:', sp2s[50:250].stats()['rms'])
print('getps  RMS:', sp0s[50:250].stats()['rms'])


The improvement of the RMS is not great, the 55 mK signal in the PS should have been more like 38 mK.

Let us subtract a baseline and compare the two spectra on the same scale.

